In [1]:
import datetime     # a libary that allows us to work with dates and times
import json         # a library that helps us make JSON format files
import numpy as np  # a library that lets us work with arrays; we import this with a new name "np"
import os           # a library that allows us access to basic operating system commands like making directories
from shapely import geometry # a library that support construction of geometry objects
import shutil       # a library that allows us access to basic operating system commands like copy
import xml.etree.ElementTree as ET # a library that allows us to work with XML files
import zipfile      # a library that allows us to unzip zip-files.
import eumdac       # a tool that helps us download via the eumetsat/data-store

In [2]:
download_dir = os.path.join(os.getcwd(), "1b_products")
os.makedirs(download_dir, exist_ok=True)

In [3]:

# load credentials
token = eumdac.AccessToken(('4gg07oXRQsrNrwskG5DqVeFQXAoa', 'L6X0YjJFcnffd9rmLHc_ZtmXsqIa'))

datastore = eumdac.DataStore(token)
for collection_id in datastore.collections:
    if "OLCI" in collection_id.title:
        print(f"Collection ID({collection_id}): {collection_id.title}")


Collection ID(EO:EUM:DAT:0556): OLCI Level 2 Ocean Colour Full Resolution (version BC003) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0409): OLCI Level 1B Full Resolution - Sentinel-3
Collection ID(EO:EUM:DAT:0407): OLCI Level 2 Ocean Colour Full Resolution - Sentinel-3
Collection ID(EO:EUM:DAT:0577): OLCI Level 1B Full Resolution (version BC002) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0557): OLCI Level 2 Ocean Colour Reduced Resolution (version BC003) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0578): OLCI Level 1B Reduced Resolution (version BC002) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0593): OLCI Level 2 Ocean Colour Reduced Resolution (version BC002) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0592): OLCI Level 2 Ocean Colour Full Resolution (version BC002) - Sentinel-3 - Reprocessed
Collection ID(EO:EUM:DAT:0410): OLCI Level 1B Reduced Resolution - Sentinel-3
Collection ID(EO:EUM:DAT:0408): OLCI Level 2 Ocean Colour Reduced Resolut

In [4]:
# set collection ID for OLCI L1 EFR
collectionID = 'EO:EUM:DAT:0409'

In [5]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(f"{selected_collection.title}\n---\n{selected_collection.abstract}")

OLCI Level 1B Full Resolution - Sentinel-3
---
OLCI (Ocean and Land Colour Instrument) Full resolution: 300m at nadir. Level 1 products are calibrated Top Of Atmosphere radiance values at OLCI 21 spectral bands. Radiances are computed from the instrument digital counts by applying geo-referencing, radiometric processing (non-linearity correction, smear correction, dark offset correction, absolute gain calibration adjusted for gain evolution with time), and stray-light correction for straylight effects in OLCI camera's spectrometer and ground imager. Additionally, spatial resampling of OLCI pixels to the 'ideal' instrument grid, initial pixel classification, and annotation at tie points with auxiliary meteorological data and acquisition geometry are provided. The radiance products are accompanied by error estimate products, however the error values are currently not available. - All Sentinel-3 NRT products are available at pick-up point in less than 3h. - All Sentinel-3 Non Time Critica

In [6]:
# time filter the collection for products
dtstart = datetime.datetime(2022, 6, 24, 0, 0)
dtend = datetime.datetime(2022, 6, 28, 23, 59)

In [7]:
products = selected_collection.search(dtstart=dtstart, dtend=dtend)
print(f"Found {len(products)} products")

Found 2311 products


In [8]:
# space/time filter the collection for products
north = 58.50
south = 53.75
east = 21.80
west = 17.50
ROI = [[west, south], [east, south], [east, north], [west, north], [west, south]]
ROI_WKT = geometry.Polygon([[p[0], p[1]] for p in ROI])

In [9]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend)
print(f"Found {len(products)} products")

Found 13 products


In [10]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend, 
    timeliness='NT')
print(f"Found {len(products)} products")

Found 13 products


In [11]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend, 
    timeliness='NT',
    sat="Sentinel-3A")
print(f"Found {len(products)} products")

Found 6 products


In [12]:
processed_list = []
final_products = []
for product in products:
    file_tags = str(product).split('_')
    file_tags = [i for i in file_tags if i]
    granule_start = file_tags[4]
    if granule_start not in processed_list:
        final_products.append(product)
        processed_list.append(granule_start)
        
print(f"Found {len(final_products)} products")

Found 5 products


In [13]:
overlap_threshold = 25

In [14]:
# 1. set up the product loop
for final_product in final_products:
    
    # 2. this reads the XML into memory and finds the correct part for the polygon and compares against our reference
    for entry in final_product.entries:
        if 'xfdumanifest.xml' in entry:
            with final_product.open(entry=entry) as fsrc:
                tree = ET.ElementTree(ET.fromstring(fsrc.data))
                root = tree.getroot()
                polygon = np.asarray(root.findall('.//gml:posList',
                    {'gml':"http://www.opengis.net/gml"})[0].text.split(' ')).astype(float)
                this_polygon = geometry.Polygon(list(zip(polygon[1::2], polygon[0::2])))
                pc_overlap = ROI_WKT.intersection(this_polygon).area/ROI_WKT.area*100

    # 3. if our overlap is greater than the threshold, we get the download the data and unzip it
    if pc_overlap > overlap_threshold:
        print(f"Percentage overlap: {int(pc_overlap)}%, downloading product")

        # download the zip file
        with final_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')

        # unzip the file
        with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith(str(final_product)):
                    zip_ref.extract(file, download_dir)
            print(f'Unzipping of product {fdst.name} finished.')

        # clean up
        os.remove(fdst.name)

Percentage overlap: 100%, downloading product
Download of product S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3.zip finished.
Unzipping of product C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3.zip finished.
Percentage overlap: 100%, downloading product
Download of product S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3.zip finished.
Unzipping of product C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3.zip finished.
Percentage overlap: 71%, downloading product


ProtocolError: ('Connection broken: IncompleteRead(420205 bytes read)', IncompleteRead(420205 bytes read))

## SNAP batch processing

In [16]:
import os           # a library that allows us access to basic operating system commands like making directories
import shutil       # a library that allows us access to basic operating system commands like copy
import subprocess   # a library that lets us call external processes
import fnmatch      # a library that supports comparing files to a specified pattern
import platform     # a library that determines the current operating system
from shapely import geometry # a library that support construction of geometry objects

In [17]:
output_dir = os.path.join(os.getcwd(), "c2rcc_outputs")
os.makedirs(output_dir, exist_ok=True)

In [18]:
template_xml = 'GPT_config_template.xml'

In [19]:
input_dir = os.path.join(os.getcwd(),'1b_products')

In [20]:
 # make a list of all the input files in your input directory
input_files=[]
for root, _, filenames in os.walk(input_dir):
    for filename in fnmatch.filter(filenames, '*xfdumanifest.xml'):
        if "EFR" in root:
            input_files.append(os.path.join(root, filename))

# and show the list        
for input_file in input_files:
    print(input_file)

C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3\xfdumanifest.xml
C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3\xfdumanifest.xml


In [21]:
print(f"This platform is: {platform.system().lower()}")
if platform.system().lower() == "darwin":
    GPT_PATH = os.path.join("/", "Applications", "snap", "bin", "gpt")
elif platform.system().lower() == "windows":
    GPT_PATH = os.path.join("C:/", "Program Files", "snap", "bin", "gpt.exe")
print(f"The default path is {GPT_PATH}, please adapt the GPT_PATH variable if this is not correct")

This platform is: windows
The default path is C:/Program Files\snap\bin\gpt.exe, please adapt the GPT_PATH variable if this is not correct


In [22]:
# set geo_region to subset
# space/time filter the collection for products
north = 59.25
west = 18.50
south = 56.75
east = 21.50

ROI = [[west, south], [east, south], [east, north], [west, north], [west, south]]
ROI_polygon = geometry.Polygon([[p[0], p[1]] for p in ROI])

In [24]:
# MAIN: the loop goes over each input file in the input_files list
for input_file in input_files:

    # 1. define an output file name. This is derived from the input file, but with a _SUBSET_IDEPIX_C2RCC suffix.
    output_file = input_file.replace(input_dir,output_dir)
    output_file = os.path.dirname(output_file).replace('.SEN3','_SUBSET_IDEPIX_C2RCC.nc')
    print(f"-------------- Processing: --------------\n{input_file}")
    print(f"-- To: --\n{os.path.basename(output_file)}")
    
    # 2. read the template xml and adapt it for the current granule
    my_config = os.path.join(output_dir, 'run_config.xml')
    print(f"-- Generating config: --\n{my_config}")
    shutil.copy(template_xml, my_config)

    with open(template_xml, 'r') as file:
        filedata = file.read()
        
    # Replace the target strings
    filedata = filedata.replace('SOURCE_PRODUCT', input_file)
    filedata = filedata.replace('OUTPUT_PRODUCT', output_file)
    filedata = filedata.replace('GEO_REGION', str(ROI_polygon))

    # Write the file out again
    with open(my_config, 'w') as file:
        file.write(filedata)

    # 3. the processing call is a follows below.
    c2rcc_processing_call = GPT_PATH + ' ' + my_config
    
    # It is useful to check that the command call is correct before launching the call
    print(f"-- Config ready; running: --\n{c2rcc_processing_call}")
    
    # Run the gpt command
    process = subprocess.Popen(c2rcc_processing_call, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, err = process.communicate()

    print(f"----- Finished running this product -----\n")

-------------- Processing: --------------
C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3\xfdumanifest.xml
-- To: --
S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002_SUBSET_IDEPIX_C2RCC.nc
-- Generating config: --
C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\c2rcc_outputs\run_config.xml
-- Config ready; running: --
C:/Program Files\snap\bin\gpt.exe C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\c2rcc_outputs\run_config.xml
----- Finished running this product -----

-------------- Processing: --------------
C:\Users\27614\Desktop\Masters Thesis\codes\Untitled Folder\1b_products\S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3\xfdumanifest.xml
-- To: --
S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_